Visualizing our Simulations
========================

Each time we run our Maestro study, we add 10 new runs of the ball bouncing simulation to our datastore. Now let's take a look at those results...

In [ ]:
from numbers import Number
from collections import defaultdict

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

import sina.datastores.sql as sina_sql
import sina.utils
from sina.datastore import create_datastore
from sina.visualization import Visualizer

# %matplotlib notebook

# use this instead of notebook if using vscode, unless you install the matplotlib extension
%matplotlib widget

# Initialization
database = 'output.sqlite'
target_type = "csv_rec"
datastore = create_datastore(database)
recs = datastore.records
vis = Visualizer(datastore)
print("Sina is ready!")

Sanity Check: are our runs randomized?
========

We want to make sure our starting positions are properly randomized. There should be a variety of values in this graph.

In [ ]:
_= vis.create_histogram("x_vel_initial", interactive=True)

Simple hypothesis testing
===============

Could there be a relationship between having a high initial velocity and a high number of bounces?

In [ ]:
_ = vis.create_scatter_plot("y_vel_initial", "num_bounces", interactive=True, matplotlib_options={"color": "orange", "alpha": 0.5, "s": 80})

Actually watching a ball bounce
===========================

Since we're collecting position at each step, we can easily visualize how the ball bounching actually works...we'll select the ball with the high number of bounces for interest.

In [ ]:
id_pool = list(recs.find_with_max("num_bounces", 1, ids_only=True))
_ = vis.create_line_plot("x_pos", "y_pos", id_pool=id_pool)

Scaling Up
==========

Each run within a set shares a few starting conditions, but differs in several others. Let's see a whole set together in 3D space to compare their performances. First, let's see what groups we have available.

In [ ]:
groups = set(x["group_id"]["value"] for x in recs.get_data(["group_id"]).values())
print("Found the following groups: {}".format(groups))

Pick whichever you like, set `GROUP_OF_INTEREST` equal to it, and watch the "fireworks"!

This 3D plot isn't (yet) a formal part of Sina, but it's simple enough.

In [ ]:
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt

# For the purposes of this demo, chose the first.
GROUP_OF_INTEREST = list(groups)[0]
print(f"Group of interest is {GROUP_OF_INTEREST}")

id_pool = list(recs.find_with_data(group_id=GROUP_OF_INTEREST))

ball_set_fig = plt.figure()
ball_set_ax = plt.axes(projection='3d')

for rec_id in id_pool:
    rec = recs.get(rec_id)
    # Some slight weirdness due to 3D axis labeling...simulation thinks of y as up-down
    ball_set_ax.plot3D(rec.data["x_pos"]["value"],
                       rec.data["z_pos"]["value"],
                       rec.data["y_pos"]["value"],
                       label=rec_id)
    ball_set_ax.set_xlabel('X')
    ball_set_ax.set_zlabel('Y')
    ball_set_ax.set_ylabel('Z')
_ = ball_set_ax.legend()

Close Curve Analysis with PyDV
=========================

While Sina allows you to survey large numbers of runs at once, it doesn't include many operations on individual curves. PyDV works directly with Sina, though, opening the door to some extremely powerful manipulations. Here, we'll perform a fourier transform to see the frequency spectrum of the ball bouncing.

TODO:
- change fourier transform out psuedocode mode
- add time

In [ ]:
# Note: pydv's big! Importing it may take a moment.
from pydv import pydvpy as pydv
import io
import json

# PyDV operates on files. We'll dump one of Sina's records to a file
PYDV_DEMO_FILENAME = "pydv_demo_sina.json"
with open(PYDV_DEMO_FILENAME, 'w') as f:
    json.dump(recs.get(rec_id).raw, f)

  
# Commented out - this returns a list and does not work
# Likely it's raw data we just need to plot with matplotlib
curves = pydv.readsina(PYDV_DEMO_FILENAME)  # raw?
print(curves)
# curve = curves.create_curve(x="time", y="y_pos")
# plt.plot(pydv.fft(curve))